In [1]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re


#Autres
import pickle
import glob
from collections import Counter
from os import path
from pathlib import Path
import glob

#Importation du pretrained model en français
nlp = spacy.load("fr_core_news_md")


In [2]:
pip install pyconll

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import des dataframes 
with open ("./Notebook_data/Exploration_Donnees_data/ArticlesEnsemble/titres_articles_full_AHMC-to-1908.txt","r") as f1:
           with open ("./Notebook_data/Exploration_Donnees_data/ArticlesEnsemble/titres_articles_full_AMN-1898-1908.txt","r") as f2:
            
            textAHMC =f1.read()
            textAMN = f2.read()

In [4]:
df_comparaison18981908 = pd.DataFrame({"AHMC":textAHMC,"AMN":textAMN},index=["1898-1908"])

In [5]:
df_comparaison18981908

,AHMC,AMN
1898-1908,Contribution à la géographie médicale. Notes s...,Rapport médical d'ensemble sur les colonnes du...


In [6]:
docAHMC = nlp(textAHMC)

In [7]:
docAMN = nlp(textAMN)

In [8]:
from spacy_conll import init_parser
import stanza
import spacy_stanza
# stanza.download('fr')
from stanza.utils.conll import CoNLL
nlpstanza = stanza.Pipeline('fr')

2021-06-06 23:04:50 INFO: Downloading default packages for language: en (English)...
2021-06-06 23:04:51 INFO: File exists: /home/humanum/stanza_resources/en/default.zip.
2021-06-06 23:04:54 INFO: Finished downloading models and saved to /home/humanum/stanza_resources.
2021-06-06 23:04:55 INFO: Downloading default packages for language: fr (French)...
2021-06-06 23:04:56 INFO: File exists: /home/humanum/stanza_resources/fr/default.zip.
2021-06-06 23:05:00 INFO: Finished downloading models and saved to /home/humanum/stanza_resources.


In [34]:
docAHMC = nlpstanza(textAHMC)

In [35]:
dicdocAHMC = docAHMC.to_dict()
conllAHMC = CoNLL.convert_dict(dicdocAHMC)
# conllAHMC

[[['1',
   'Contribution',
   'contribution',
   'NOUN',
   '_',
   'Gender=Fem|Number=Sing',
   '0',
   'root',
   '_',
   'start_char=0|end_char=12'],
  ['2',
   'à',
   'à',
   'ADP',
   '_',
   '_',
   '4',
   'case',
   '_',
   'start_char=13|end_char=14'],
  ['3',
   'la',
   'le',
   'DET',
   '_',
   'Definite=Def|Gender=Fem|Number=Sing|PronType=Art',
   '4',
   'det',
   '_',
   'start_char=15|end_char=17'],
  ['4',
   'géographie',
   'géographie',
   'NOUN',
   '_',
   'Gender=Fem|Number=Sing',
   '1',
   'nmod',
   '_',
   'start_char=18|end_char=28'],
  ['5',
   'médicale',
   'médical',
   'ADJ',
   '_',
   'Gender=Fem|Number=Sing',
   '4',
   'amod',
   '_',
   'start_char=29|end_char=37'],
  ['6',
   '.',
   '.',
   'PUNCT',
   '_',
   '_',
   '1',
   'punct',
   '_',
   'start_char=37|end_char=38']],
 [['1',
   'Notes',
   'note',
   'NOUN',
   '_',
   'Gender=Fem|Number=Plur',
   '0',
   'root',
   '_',
   'start_char=39|end_char=44'],
  ['2',
   'sur',
   'sur',
   '

In [36]:
def count_words(doc):
    """
    Compte la fréquence de chaque mot du texte donné en argument
    texte au format conll
    """
    words_freq = Counter()
#     with open(filepath) as doc:
#         print(type(doc))
    for line in doc:
        cols = line.split("\t")
        if len(cols) > 4:
            form = cols[1]
            lemma = cols[2]
            pos = cols[3]
            if pos != "PONCT":
                words_freq[form] += 1
    return words_freq

In [37]:
count_words(conllAHMC)

AttributeError: 'list' object has no attribute 'split'

In [15]:
counts = dict() # dictionnaire {doc: counter mots}
vocab = set() # vocabulaire du corpus  -> cf le NB ci-dessous
for doc in glob.glob('./Notebook_data/Exploration_Donnees_data/ArticlesEnsemble/'):
    doc_name = path.splitext(path.basename(doc))[0]
    counts[doc_name] = count_words(doc) ### on utilise ici la fonction définie au dessus pour intégrer les résultats au dico
    vocab = vocab.union(set(counts[doc_name]))
print("Nombre de types de chaque roman du corpus :")
total_words = 0
for doc in counts:
    print(f"{doc} : {len(counts[doc])}")
    total_words += sum(counts[doc].values())
print(f"\nTaille du vocabulaire du corpus : {len(vocab)}")
print(f"Nombre de mots du corpus : {total_words}")

IsADirectoryError: [Errno 21] Is a directory: './Notebook_data/Exploration_Donnees_data/ArticlesEnsemble/'

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=text, window=5, min_count=1, workers=4)
model.save("word2vec.model")


In [ ]:
model

In [ ]:
import stanza
from spacy_conll import init_parser


# Initialise English parser, already including the ConllFormatter as a pipeline component.
# Indicate that we want to get the CoNLL headers in the string output.
# `use_gpu` and `verbose` are specific to stanza (and stanfordnlp). These keywords arguments
# are passed onto their Pipeline() initialisation
nlp = init_parser("stanza","fr",
                  parser_opts={"use_gpu": True, "verbose": False},
                  include_headers=True)
# Parse a given string
doc = nlp("A cookie is a baked or cooked food that is typically small, flat and sweet. It usually contains flour, sugar and some type of oil or fat.")

# Get the CoNLL representation of the whole document, including headers
conll = doc._.conll_str
print(conll)